In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
# data loading in appropriate formate

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


11501568/11490434 [==============================] - 0s 0us/step


In [3]:
y_train=tf.keras.utils.to_categorical(y_train)

In [4]:
class CONV(tf.keras.layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CONV, self).__init__()
        self.conv = tf.keras.layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor,training=False):
        x = self.conv(input_tensor)
        x = self.bn(x,training=training)
        x = tf.nn.relu(x)
        return x

In [5]:
class RES_block(tf.keras.layers.Layer):
    def __init__(self):
        super(RES_block,self).__init__()
        self.conv1=CONV(20,3)
        self.conv2=CONV(30,3)
        self.conv3=CONV(30,3)
        self.identity_mapping = CONV(30, 1)
        self.max=tf.keras.layers.MaxPool2D(3)

    def call(self, input_tensor,training=False):
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.conv3(x + self.identity_mapping(input_tensor), training=training)
        x = self.max(x)
        return x
        


In [8]:
#model subclassing with convolution layer.

class MyModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        # define all layers in init
        # Layer of Block 1
        self.r1 = RES_block()

        #fully connected layer
        self.flat = tf.keras.layers.Flatten()
        self.d1 = keras.layers.Dense(20, activation='relu')
        self.d2 = keras.layers.Dense(num_classes)


    def call(self, input_tensor):
        # forward pass: block 1 
        x = self.r1(input_tensor)

        # forward pass: FC layer
        x=self.flat(x)
        x = self.d1(x)
        return self.d2(x)


In [9]:
model=MyModel(10)

"""loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])"""
model.compile(
          loss      = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
          metrics   = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())

In [10]:
model.build(input_shape=(None, 28, 28, 1))
model.call(tf.keras.Input(shape=(28, 28, 1)))
model.summary()
#no of parameter in conv  = ((m*n*d)+1)*k) for ex ((3*3*1)+1)*32 = 320
# ((3*3*32)+1)*64) = 18496

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block (RES_block)       (None, 9, 9, 30)          14260     
                                                                 
 flatten (Flatten)           (None, 2430)              0         
                                                                 
 dense (Dense)               (None, 20)                48620     
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
Total params: 63,090
Trainable params: 62,870
Non-trainable params: 220
_________________________________________________________________


In [11]:
model.fit(x_train, y_train, batch_size=128,epochs=2,validation_split=0.2)

Epoch 1/2
375/375 [==============================] - 228s 605ms/step - loss: 0.3021 - categorical_accuracy: 0.9074 - val_loss: 0.2487 - val_categorical_accuracy: 0.9658
Epoch 2/2
375/375 [==============================] - 228s 607ms/step - loss: 0.0767 - categorical_accuracy: 0.9767 - val_loss: 0.0764 - val_categorical_accuracy: 0.9782
